# H20.Ai

In [1]:
from boto.s3.connection import Location
from boto.s3.key import Key
from configparser import ConfigParser
from sklearn import metrics
from sklearn import svm
from sklearn.cross_validation  import cross_val_score
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.neural_network import MLPClassifier
from statsmodels.formula.api import logit, probit, poisson, ols
import boto
import datetime
import glob
import logging
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
import shutil
import statsmodels.api as sm
import time
import zipfile
import requests

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
    config = ConfigParser()

    config_file = os.path.join(os.path.dirname('__file__'), 'config.ini')
    config.read(config_file)
    default = config['user.data']
    user=default['username']
    passwd=default['password']
    trainQ=default['trainQtr']
    testQ=default['testQtr']

    print("USERNAME=" + user)
    print("PASSWORD=" + passwd)
    print("TRAINQUARTER=" + (trainQ))
    print("TESTQUARTER=" + (testQ))

USERNAME=yadav.ank@Husky.neu.edu
PASSWORD=OAt;lsKF
TRAINQUARTER=Q12005
TESTQUARTER=Q22005


In [3]:
df=pd.read_csv('trainData_' + trainQ + '.csv')
traincols=['MONTHLY_REPORTING_PERIOD','CURRENT_ACTUAL_UPB','LOAN_AGE',
           'REM_MTH_LEGAL_MATURITY','CURRENT_INTEREST_DATE','CURRENT_DEFERRED_UPB']
y_train=df['DELINQUENT']
Train_DF=df[traincols]
Train_DF.head()


,MONTHLY_REPORTING_PERIOD,CURRENT_ACTUAL_UPB,LOAN_AGE,REM_MTH_LEGAL_MATURITY,CURRENT_INTEREST_DATE,CURRENT_DEFERRED_UPB
0,200504,190000.0,0,360,5.625,0.0
1,200505,190000.0,1,359,5.625,0.0
2,200506,190000.0,2,358,5.625,0.0
3,200507,189000.0,3,357,5.625,0.0
4,200508,189000.0,4,356,5.625,0.0


In [4]:
test_df=pd.read_csv('testData_' + testQ + '.csv')
testcols = ['MONTHLY_REPORTING_PERIOD', 'CURRENT_ACTUAL_UPB', 'LOAN_AGE',
                'REM_MTH_LEGAL_MATURITY', 'CURRENT_INTEREST_DATE', 'CURRENT_DEFERRED_UPB']
y_test = test_df['DELINQUENT']
Test_DF = test_df[testcols]
Test_DF.head()

,MONTHLY_REPORTING_PERIOD,CURRENT_ACTUAL_UPB,LOAN_AGE,REM_MTH_LEGAL_MATURITY,CURRENT_INTEREST_DATE,CURRENT_DEFERRED_UPB
0,200507,214000.0,0,360,5.75,0.0
1,200508,214000.0,1,359,5.75,0.0
2,200509,214000.0,2,358,5.75,0.0
3,200510,213000.0,3,357,5.75,0.0
4,200511,213000.0,4,356,5.75,0.0


In [5]:
import h2o

In [6]:
h2o.init(nthreads = -1, max_mem_size = 8)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.171-b11, mixed mode)
  Starting server from C:\ProgramData\Anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\ankit\AppData\Local\Temp\tmpq_xdl4a5
  JVM stdout: C:\Users\ankit\AppData\Local\Temp\tmpq_xdl4a5\h2o_ankit_started_from_python.out
  JVM stderr: C:\Users\ankit\AppData\Local\Temp\tmpq_xdl4a5\h2o_ankit_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,07 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.0.2
H2O cluster version age:,8 days
H2O cluster name:,H2O_from_python_ankit_9jnaus
H2O cluster total nodes:,1
H2O cluster free memory:,7.111 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


# GLM(Generalized Linear Model)-
In statistics, the generalized linear model (GLM) is a 
flexible generalization of ordinary linear regression that allows for 
response variables that have error distribution models other than a normal distribution.
-Wikipedia

In [7]:
# Import H2O GLM:
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

In [8]:
# Initialize the GLM estimator:
# Similar to R's glm() and H2O's R GLM, H2O's GLM has the "family" argument

glm_fit1 = H2OGeneralizedLinearEstimator(family='binomial', model_id='glm_fit1')

In [12]:
data = h2o.import_file('trainData_' + trainQ + '.csv')  
splits = data.split_frame(ratios=[0.9, 0.05], seed=1)  



Parse progress: |█████████████████████████████████████████████████████████| 100%


In [14]:
train = splits[0]
valid = splits[1]
data = h2o.import_file('testData_' + testQ + '.csv')  
splits = data.split_frame(ratios=[0.9, 0.05], seed=1) 
test = splits[0]
y='DELINQUENT'
x=train.columns
glm_fit1.train(x=x, y=y, training_frame=train)

Parse progress: |█████████████████████████████████████████████████████████| 100%
glm Model Build progress: |███████████████████████████████████████████████| 100%


In [15]:
glm_fit2 = H2OGeneralizedLinearEstimator(family='binomial', model_id='glm_fit2', lambda_search=True)
glm_fit2.train(x=x, y=y, training_frame=train, validation_frame=valid)

glm Model Build progress: |███████████████████████████████████████████████| 100%


In [16]:
glm_perf1 = glm_fit1.model_performance(test)
glm_perf2 = glm_fit2.model_performance(test)

In [17]:
# Print model performance
print(glm_perf1)
print(glm_perf2)


ModelMetricsBinomialGLM: glm
** Reported on test data. **

MSE: 0.04373221130262242
RMSE: 0.20912247919012059
LogLoss: 0.17690328947718295
Null degrees of freedom: 2249806
Residual degrees of freedom: 2249799
Null deviance: 859286.2676139451
Residual deviance: 795996.5179775851
AIC: 796012.5179775851
AUC: 0.7263026958466243
pr_auc: 0.12097318430066503
Gini: 0.45260539169324865
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.06725747456685717: 


,0,1,Error,Rate
0,1793771.0,349259.0,0.163,(349259.0/2143030.0)
1,61935.0,44842.0,0.58,(61935.0/106777.0)
Total,1855706.0,394101.0,0.1828,(411194.0/2249807.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.0672575,0.1790536,252.0
max f2,0.0473025,0.2997437,293.0
max f0point5,0.1655558,0.1681893,139.0
max accuracy,0.7170147,0.9525542,1.0
max precision,0.8212219,1.0,0.0
max recall,0.0056712,1.0,399.0
max specificity,0.8212219,1.0,0.0
max absolute_mcc,0.0477661,0.1550690,292.0
max min_per_class_accuracy,0.0448084,0.6704792,299.0
max mean_per_class_accuracy,0.0411214,0.6750782,308.0


Gains/Lift Table: Avg response rate:  4.75 %, avg score:  4.75 %



...

,0,1,Error,Rate
0,1749818.0,393212.0,0.1835,(393212.0/2143030.0)
1,58084.0,48693.0,0.544,(58084.0/106777.0)
Total,1807902.0,441905.0,0.2006,(451296.0/2249807.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.0629480,0.1774908,257.0
max f2,0.0466500,0.3012265,294.0
max f0point5,0.1718812,0.1635409,119.0
max accuracy,0.6095426,0.9525373,0.0
max precision,0.3972644,0.3120176,24.0
max recall,0.0061709,1.0,399.0
max specificity,0.6095426,0.9999977,0.0
max absolute_mcc,0.0466500,0.1565728,294.0
max min_per_class_accuracy,0.0452405,0.6737722,298.0
max mean_per_class_accuracy,0.0404879,0.6784285,311.0


Gains/Lift Table: Avg response rate:  4.75 %, avg score:  4.75 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0100004,0.2061513,5.5393535,5.5393535,0.2629006,0.2945943,0.2629006,0.2945943,0.0553958,0.0553958,453.9353515,453.9353515
,2,0.0200004,0.1594262,2.9959729,4.2676915,0.1421904,0.1764791,0.2025468,0.2355380,0.0299596,0.0853555,199.5972872,326.7691455
,3,0.0300004,0.1427860,2.3329067,3.6227727,0.1107210,0.1498676,0.1719387,0.2069816,0.0233290,0.1086845,133.2906666,262.2772747
,4,0.0400003,0.1324222,1.9901352,3.2146179,0.0944528,0.1375942,0.1525674,0.1896350,0.0199013,0.1285857,99.0135152,221.4617884
,5,0.0500003,0.1220437,2.2102207,3.0137402,0.1048982,0.1268839,0.1430337,0.1770849,0.0221021,0.1506879,121.0220687,201.3740230
,6,0.1000001,0.0899342,2.1622701,2.5880071,0.1026225,0.1043797,0.1228281,0.1407324,0.1081132,0.2588011,116.2270136,158.8007076
,7,0.1500000,0.0727449,2.1431648,2.4397268,0.1017157,0.0806020,0.1157907,0.1206890,0.1071579,0.3659590,114.3164839,143.9726769
,8,0.2000003,0.0620981,1.9219384,2.3102791,0.0912162,0.0670398,0.1096470,0.1072767,0.0960975,0.4620564,92.1938378,131.0279096
,9,0.3000000,0.0491038,1.5612961,2.0606189,0.0740999,0.0549962,0.0977980,0.0898499,0.1561291,0.6181856,56.1296147,106.0618853
,10,0.4000001,0.0407144,1.2163652,1.8495552,0.0577293,0.0445927,0.0877808,0.0785356,0.1216367,0.7398222,21.6365199,84.9555205


In [18]:
# Retreive test set AUC
print(glm_perf1.auc())
print(glm_perf2.auc())

0.7263026958466243
0.725242631235037


AUC is an abbrevation for area under the curve. 
It is used in classification analysis in order to determine which of the used models predicts the classes best.

In [19]:
# Compare test AUC to the training AUC and validation AUC
print(glm_fit2.auc(train=True))
print(glm_fit2.auc(valid=True))

0.725242631235037
0.7247373992219532


# H2ORandomForestEstimator

In [20]:
# Import H2O RF:
from h2o.estimators.random_forest import H2ORandomForestEstimator

In [21]:
# Initialize the RF estimator:

rf_fit1 = H2ORandomForestEstimator(model_id='rf_fit1', seed=1)

In [22]:
rf_fit1.train(x=x, y=y, training_frame=train)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [23]:
rf_fit2 = H2ORandomForestEstimator(model_id='rf_fit2', ntrees=100, seed=1)
rf_fit2.train(x=x, y=y, training_frame=train)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [24]:
rf_perf1 = rf_fit1.model_performance(test)
rf_perf2 = rf_fit2.model_performance(test)

In [25]:
rf_fit3 = H2ORandomForestEstimator(model_id='rf_fit3', seed=1, nfolds=5)
rf_fit3.train(x=x, y=y, training_frame=data)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [26]:
print(rf_fit3)

Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  rf_fit3


ModelMetricsRegression: drf
** Reported on train data. **

MSE: 0.029892483406180913
RMSE: 0.17289442849953526
MAE: 0.06652812980436693
RMSLE: 0.1179388649954172
Mean Residual Deviance: 0.029892483406180913

ModelMetricsRegression: drf
** Reported on cross-validation data. **

MSE: 0.029864916490598168
RMSE: 0.1728146882952898
MAE: 0.06682284371969262
RMSLE: 0.11762865766378246
Mean Residual Deviance: 0.029864916490598168
Cross-Validation Metrics Summary: 


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
mae,0.066823,0.0001805,0.0668966,0.0665216,0.0672710,0.0667611,0.0666647
mean_residual_deviance,0.0298651,0.0001383,0.0300506,0.0296694,0.0301413,0.0297907,0.0296734
mse,0.0298651,0.0001383,0.0300506,0.0296694,0.0301413,0.0297907,0.0296734
r2,0.3393885,0.0029952,0.3388648,0.3426735,0.3316120,0.3402234,0.3435686
residual_deviance,0.0298651,0.0001383,0.0300506,0.0296694,0.0301413,0.0297907,0.0296734
rmse,0.1728142,0.0004000,0.1733511,0.1722481,0.1736125,0.1725998,0.1722596
rmsle,0.1176281,0.0003118,0.1179830,0.1172539,0.1183056,0.1174224,0.1171758


Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
,2018-11-30 09:00:29,21 min 17.944 sec,0.0,nan,nan,nan
,2018-11-30 09:00:33,21 min 21.827 sec,1.0,0.1938197,0.0683119,0.0375661
,2018-11-30 09:00:40,21 min 28.798 sec,3.0,0.1865978,0.0662517,0.0348187
,2018-11-30 09:00:47,21 min 35.772 sec,5.0,0.1847681,0.0671518,0.0341393
,2018-11-30 09:00:54,21 min 42.988 sec,7.0,0.1817478,0.0669436,0.0330323
---,---,---,---,---,---,---
,2018-11-30 09:03:05,23 min 53.663 sec,42.0,0.1732633,0.0666030,0.0300202
,2018-11-30 09:03:11,24 min 0.425 sec,44.0,0.1731099,0.0665535,0.0299670
,2018-11-30 09:03:18,24 min 7.304 sec,46.0,0.1731081,0.0665776,0.0299664
,2018-11-30 09:03:25,24 min 14.181 sec,48.0,0.1730604,0.0665934,0.0299499



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
LOAN_SEQ_NO,368430.0625000,1.0,0.3021344
CURRENT_ACTUAL_UPB,182531.7343750,0.4954312,0.1496868
REM_MTH_LEGAL_MATURITY,175378.5312500,0.4760158,0.1438207
C1,141173.75,0.3831765,0.1157708
CURRENT_INTEREST_DATE,130796.3125000,0.3550099,0.1072607
LOAN_AGE,127015.8906250,0.3447490,0.1041605
MONTHLY_REPORTING_PERIOD,81796.625,0.2220140,0.0670781
CURRENT_DEFERRED_UPB,12301.5693359,0.0333892,0.0100880
